In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import accuracy_score, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
import plotly.express as px

In [ ]:
ratings = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t",low_memory=False)
actorslist = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t",low_memory=False)
title_akas = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz", sep="\t",low_memory=False)
name_basics = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep="\t",low_memory=False)
title_basic = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t",low_memory=False)

In [ ]:
actors_male_female = actorslist.loc[actorslist["category"].str.contains("director|actress|actor")]

In [ ]:
actors = pd.merge(actors_male_female,
                       name_basics,
                       how = "inner",
                       left_on = "nconst",
                       right_on = "nconst")

In [ ]:
imdbmovs2 = pd.merge(imdbmovs,
                       actors,
                       how = "inner",
                       left_on = "tconst",
                       right_on = "tconst")

In [ ]:
# Filter movies
title_basic_mov = title_basic[(title_basic["titleType"]== "movie") & (title_basic["isAdult"]=="0")]
title_basic_mov.head(20)

In [ ]:
mov_ratings = pd.merge(title_basic_mov,
                       ratings,
                       how = "inner",
                       left_on = "tconst",
                       right_on = "tconst")
mov_ratings.head(20)

In [ ]:
title_aka_fr = title_akas[title_akas["region"] == "FR"]
title_aka_fr.head(20)

In [ ]:
mov_update = pd.merge(mov_ratings,
                       title_aka_fr,
                       how = "inner",
                       left_on = "tconst",
                       right_on = "titleId")

In [ ]:
mov_update.drop("titleId",
                axis = 1,
                inplace = True)
mov_update.drop("ordering",
                axis = 1,
                inplace = True)

In [ ]:
mov_update.head(10)

In [ ]:
mov_update["tconst"].value_counts()

In [ ]:
imdbmovs = mov_update.loc[mov_update["types"]=="imdbDisplay"]
imdbmovs["tconst"].value_counts()

In [ ]:
imdbmovs.drop_duplicates(subset="tconst",inplace=True)

In [ ]:
imdbmovs["tconst"].value_counts()

In [ ]:
imdbmovs.describe()

In [ ]:
imdbmovs["runtimeMinutes"] = pd.to_numeric(imdbmovs['runtimeMinutes'], errors='coerce')

In [ ]:
imdbmovs.describe()

In [ ]:
imdbmovs["runtimeMinutes"].isna()

In [ ]:
imdbmovs["runtimeMinutes"].fillna(imdbmovs["runtimeMinutes"].mean(),
                               inplace=True)

In [ ]:
imdbmovs.head(20)

In [ ]:
imdbmovs2.head(10)

In [ ]:
imdbmovs2.info()

In [ ]:
colonnes = ["job","endYear","isAdult","region","language","types","attributes","isOriginalTitle","titleType"]
imdbmovs2.drop(colonnes,
                axis = 1,
                inplace = True)

In [ ]:
imdbmovs2.info()

In [ ]:
imdbmovsfilter1 = imdbmovs2.loc[(imdbmovs2["averageRating"]>=6) & (imdbmovs2["runtimeMinutes"]>=60) & (imdbmovs2["numVotes"]>=10000)]

In [ ]:
imdbmovsfilter1.sort_values(by="startYear",ascending=False)

In [ ]:
imdbmovsfilter1["category"].value_counts()

In [ ]:
import plotly.graph_objects as go

labels = ['Actor','Actress']
values = [6996, 3696]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, title = "Répartition Hommes/Femmes", pull=[0.1, 0.1])])
fig.show()

In [ ]:
imdbmovsfilter1["tconst"].value_counts()

In [ ]:
imdbmovs.to_csv("Avec_tconst.csv",sep=",",index=False)

In [ ]:
imdbmovsfilter1.to_csv("Avec_nconst.csv",sep=",",index=False)

In [ ]:
imdbmovs2.info()

In [ ]:
tconstmovs = imdbmovs.loc[(imdbmovs["averageRating"]>=6) & (imdbmovs["runtimeMinutes"]>=60) & (imdbmovs["numVotes"]>=10000)]
tconstmovs["genres"].str.get_dummies(sep=",").sum().sort_values(ascending=False).head(10)

In [38]:
nconstmovs = pd.read_csv(r"C:\Users\Tony\Desktop\PROJET2\Avec_nconst.csv")

In [39]:
nconstmovs = nconstmovs.loc[(nconstmovs["averageRating"]>=6) & (nconstmovs["runtimeMinutes"]>=60) & (nconstmovs["numVotes"]>=10000)]

In [40]:
nconstmovs.sort_values(by="averageRating",ascending=False)

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,nconst,category,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
6438,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2738697,Les Évadés,1,nm0000209,actor,"[""Andy Dufresne""]",Tim Robbins,1958,\N,"actor,producer,director","tt0105151,tt0091225,tt0327056,tt0112818"
6439,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2738697,Les Évadés,4,nm0006669,actor,"[""Heywood""]",William Sadler,1950,\N,"actor,soundtrack,producer","tt0884328,tt0111161,tt0099423,tt0101452"
6436,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2738697,Les Évadés,3,nm0348409,actor,"[""Warden Norton""]",Bob Gunton,1945,\N,"actor,soundtrack","tt0111161,tt3322312,tt0285331,tt3305388"
6435,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2738697,Les Évadés,2,nm0000151,actor,"[""Ellis Boyd 'Red' Redding""]",Morgan Freeman,1937,\N,"actor,producer,director","tt0405159,tt1057500,tt0114369,tt0097239"
6437,tt0111161,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2738697,Les Évadés,5,nm0001104,director,\N,Frank Darabont,1959,\N,"writer,producer,director","tt1520211,tt0884328,tt0111161,tt0120689"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9149,tt0318974,The Alamo,The Alamo,2004,137.0,"Drama,History,War",6.0,21597,Alamo,1,nm0000598,actor,"[""Sam Houston""]",Dennis Quaid,1954,\N,"actor,producer,soundtrack","tt0319262,tt0186151,tt0297884,tt0093260"
9150,tt0318974,The Alamo,The Alamo,2004,137.0,"Drama,History,War",6.0,21597,Alamo,3,nm0248408,actor,"[""Antonio Lopez de Santa Ana""]",Emilio Echevarría,\N,\N,actor,"tt0245574,tt0245712,tt0246460,tt0449467"
9151,tt0318974,The Alamo,The Alamo,2004,137.0,"Drama,History,War",6.0,21597,Alamo,4,nm0000574,actor,"[""James Bowie""]",Jason Patric,1966,\N,"actor,producer","tt0120179,tt0480255,tt0117665,tt0093437"
9152,tt0318974,The Alamo,The Alamo,2004,137.0,"Drama,History,War",6.0,21597,Alamo,2,nm0000671,actor,"[""Davy Crockett""]",Billy Bob Thornton,1955,\N,"actor,writer,soundtrack","tt0120324,tt0243133,tt0120591,tt0117666"


In [53]:
actors = pd.DataFrame(nconstmovs.loc[nconstmovs['category'].str.contains('actor|actress')])

In [100]:
top1actor = actors.loc[actors["ordering"]==1]

In [101]:
top1 = top1actor["nconst"].str.get_dummies()

In [107]:
top1

,nm0000001,nm0000003,nm0000004,nm0000006,nm0000007,nm0000008,nm0000009,nm0000010,nm0000011,nm0000012,...,nm9602952,nm9665672,nm9760132,nm9858131,nm9862858,nm9877392,nm9903838,nm9912940,nm9913973,nm9930181
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
algotest2 = pd.concat([top1actor,
top1],axis=1)

In [111]:
genresdum = algotest2["genres"].str.get_dummies(sep=",")

In [112]:
algotest3 = pd.concat([algotest2,
genresdum],axis=1)
algotest3

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
4,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,1,...,0,0,0,0,0,0,0,0,1,0
8,tt0006864,Intolerance,Intolerance: Love's Struggle Throughout the Ages,1916,163.0,"Drama,History",7.7,16121,Intolérance,1,...,0,0,0,0,0,0,0,0,0,0
10,tt0012349,The Kid,The Kid,1921,68.0,"Comedy,Drama,Family",8.3,129088,Le Kid,1,...,0,0,0,0,0,0,0,0,0,0
21,tt0013257,Häxan,Häxan,1922,91.0,"Documentary,Fantasy,Horror",7.6,15426,La sorcellerie à travers les âges,1,...,1,0,0,0,0,0,0,0,0,0
24,tt0013427,Nanook of the North,Nanook of the North,1922,78.0,Documentary,7.6,12886,Nanouk l'Esquimau,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27056,tt9851854,Major,Major,2022,150.0,"Action,Biography,Drama",8.1,30356,Major,1,...,0,0,0,0,0,0,0,0,0,0
27062,tt9866072,Holidate,Holidate,2020,104.0,"Comedy,Romance",6.1,71605,Holidate,1,...,0,0,0,0,1,0,0,0,0,0
27066,tt9893250,I Care a Lot,I Care a Lot,2020,118.0,"Comedy,Crime,Drama",6.3,137616,I Care a Lot,1,...,0,0,0,0,0,0,0,0,0,0
27071,tt9900782,Kaithi,Kaithi,2019,145.0,"Action,Adventure,Crime",8.4,35885,Kaithi,1,...,0,0,0,0,0,0,0,0,0,0


In [105]:
nconstdf = pd.concat([top1actor,
                     dfalgo])

In [106]:
nconstdf

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,...,nm9602952,nm9665672,nm9760132,nm9858131,nm9862858,nm9877392,nm9903838,nm9912940,nm9913973,nm9930181
4,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,tt0006864,Intolerance,Intolerance: Love's Struggle Throughout the Ages,1916,163.0,"Drama,History",7.7,16121,Intolérance,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,tt0012349,The Kid,The Kid,1921,68.0,"Comedy,Drama,Family",8.3,129088,Le Kid,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,tt0013257,Häxan,Häxan,1922,91.0,"Documentary,Fantasy,Horror",7.6,15426,La sorcellerie à travers les âges,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,tt0013427,Nanook of the North,Nanook of the North,1922,78.0,Documentary,7.6,12886,Nanouk l'Esquimau,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27056,tt9851854,Major,Major,2022,150.0,"Action,Biography,Drama",8.1,30356,Major,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27062,tt9866072,Holidate,Holidate,2020,104.0,"Comedy,Romance",6.1,71605,Holidate,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27066,tt9893250,I Care a Lot,I Care a Lot,2020,118.0,"Comedy,Crime,Drama",6.3,137616,I Care a Lot,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27071,tt9900782,Kaithi,Kaithi,2019,145.0,"Action,Adventure,Crime",8.4,35885,Kaithi,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
top2actor = actors.loc[actors["ordering"]==2]

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,nconst,category,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
3,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,2,nm0550615,actress,"[""Flora Cameron - The Pet Sister""]",Mae Marsh,1894,1968,actress,"tt0041503,tt0004972,tt0006826,tt0006864"
5,tt0006864,Intolerance,Intolerance: Love's Struggle Throughout the Ages,1916,163.0,"Drama,History",7.7,16121,Intolérance,2,nm0366008,actor,"[""The Boy""]",Robert Harron,1893,1920,"actor,art_department,cinematographer","tt0006864,tt0006826,tt0008379,tt0009559"
11,tt0012349,The Kid,The Kid,1921,68.0,"Comedy,Drama,Family",8.3,129088,Le Kid,2,nm0701012,actress,"[""The Woman""]",Edna Purviance,1895,1958,actress,"tt0008133,tt0017575,tt0014624,tt0012349"
22,tt0013257,Häxan,Häxan,1922,91.0,"Documentary,Fantasy,Horror",7.6,15426,La sorcellerie à travers les âges,2,nm0159770,actress,"[""En ældre bondekone"",""Older Farm Lady""]",Elisabeth Christensen,1855,1923,actress,"tt3126502,tt2409466,tt0013257,tt2350494"
25,tt0013427,Nanook of the North,Nanook of the North,1922,78.0,Documentary,7.6,12886,Nanouk l'Esquimau,2,nm0638725,actress,"[""Nanook's Wife - the Smiling One""]",Alice Nevalinga,\N,\N,NaN,tt0013427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27055,tt9851854,Major,Major,2022,150.0,"Action,Biography,Drama",8.1,30356,Major,2,nm0695177,actor,"[""K. Unnikrishnan""]",Prakash Raj,1965,\N,"actor,producer,director","tt0376127,tt0140303,tt15097216,tt1526875"
27063,tt9866072,Holidate,Holidate,2020,104.0,"Comedy,Romance",6.1,71605,Holidate,2,nm3478396,actor,"[""Jackson""]",Luke Bracey,1989,\N,actor,"tt1972779,tt2058673,tt2402157,tt1583421"
27067,tt9893250,I Care a Lot,I Care a Lot,2020,118.0,"Comedy,Crime,Drama",6.3,137616,I Care a Lot,2,nm0227759,actor,"[""Roman Lunyov""]",Peter Dinklage,1969,\N,"actor,producer,soundtrack","tt0944947,tt0340377,tt5027774,tt1877832"
27070,tt9900782,Kaithi,Kaithi,2019,145.0,"Action,Adventure,Crime",8.4,35885,Kaithi,2,nm1230844,actor,"[""Bejoy""]",Narain,1979,\N,actor,"tt9900782,tt2367996,tt1922545,tt9179430"


In [67]:
top2 = top2actor["nconst"].str.get_dummies()

In [69]:
pd.concat([top1,
          top2],
          axis=1)

,nm0000001,nm0000003,nm0000004,nm0000006,nm0000007,nm0000008,nm0000009,nm0000010,nm0000011,nm0000012,...,nm9820630,nm9826817,nm9858130,nm9862859,nm9875384,nm9891874,nm9912941,nm9928618,nm9964894,nm9979692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
directors = pd.DataFrame(nconstmovs.loc[nconstmovs['category'] == 'director'])

In [45]:
directors["nconst"] = directors["primaryName"]

In [49]:
directors.drop(["primaryName","birthYear","deathYear"],axis=1,inplace=True)

In [63]:
directors

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,nconst,category,characters,primaryProfession,knownForTitles
0,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,5,D.W. Griffith,director,\N,"director,writer,producer","tt0006864,tt0004972,tt0009968,tt0010484"
6,tt0006864,Intolerance,Intolerance: Love's Struggle Throughout the Ages,1916,163.0,"Drama,History",7.7,16121,Intolérance,5,D.W. Griffith,director,\N,"director,writer,producer","tt0006864,tt0004972,tt0009968,tt0010484"
28,tt0013427,Nanook of the North,Nanook of the North,1922,78.0,Documentary,7.6,12886,Nanouk l'Esquimau,5,Robert J. Flaherty,director,\N,"director,writer,producer","tt0028827,tt0025456,tt0017162,tt0040550"
30,tt0013442,Nosferatu,"Nosferatu, eine Symphonie des Grauens",1922,94.0,"Fantasy,Horror",7.9,100603,Nosferatu le vampire,5,F.W. Murnau,director,\N,"director,writer,producer","tt0013442,tt0015064,tt0022458,tt0018455"
34,tt0014341,Our Hospitality,Our Hospitality,1923,65.0,"Comedy,Romance,Thriller",7.8,11851,Les lois de l'hospitalité,5,John G. Blystone,director,\N,"director,miscellaneous,producer","tt0014341,tt0020689,tt0017631,tt0021138"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27060,tt9866072,Holidate,Holidate,2020,104.0,"Comedy,Romance",6.1,71605,Holidate,5,John Whitesell,director,\N,"director,producer","tt9866072,tt0044265,tt1464174,tt0790604"
27064,tt9883832,Moonage Daydream,Moonage Daydream,2022,135.0,"Biography,Documentary,History",7.7,11140,Moonage Daydream,5,Brett Morgen,director,\N,"producer,director,writer","tt0303353,tt4229236,tt7207238"
27068,tt9893250,I Care a Lot,I Care a Lot,2020,118.0,"Comedy,Crime,Drama",6.3,137616,I Care a Lot,5,J Blakeson,director,\N,"director,writer,actor","tt6128262,tt9893250,tt2304933,tt1379177"
27074,tt9900782,Kaithi,Kaithi,2019,145.0,"Action,Adventure,Crime",8.4,35885,Kaithi,5,Lokesh Kanagaraj,director,\N,"writer,director,actor","tt9900782,tt10579952,tt9179430,tt5559528"


In [17]:
nconstmovs["nconst"] = nconstmovs["primaryName"]

In [20]:
columns = ["primaryTitle","originalTitle","ordering","category","characters","primaryName","birthYear","deathYear","primaryProfession","knownForTitles"]

In [21]:
nconstmovs.drop(columns,axis=1,inplace=True)

In [22]:
nconstmovs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27080 entries, 0 to 27079
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          27080 non-null  object 
 1   startYear       27080 non-null  int64  
 2   runtimeMinutes  27080 non-null  float64
 3   genres          27080 non-null  object 
 4   averageRating   27080 non-null  float64
 5   numVotes        27080 non-null  int64  
 6   title           27080 non-null  object 
 7   nconst          27080 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.9+ MB


In [70]:
nconstmovs

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,nconst,category,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,5,nm0000428,director,\N,D.W. Griffith,1875,1948,"director,writer,producer","tt0006864,tt0004972,tt0009968,tt0010484"
1,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,3,nm0910400,actor,"[""Col. Ben Cameron aka The Little Colonel""]",Henry B. Walthall,1878,1936,"actor,assistant_director","tt0007478,tt0008198,tt0018097,tt0022753"
2,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,4,nm0178270,actress,"[""Margaret Cameron - The Elder Sister""]",Miriam Cooper,1891,1976,"actress,casting_director,costume_department","tt0011102,tt0006864,tt0012665,tt0012515"
3,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,2,nm0550615,actress,"[""Flora Cameron - The Pet Sister""]",Mae Marsh,1894,1968,actress,"tt0041503,tt0004972,tt0006826,tt0006864"
4,tt0004972,The Birth of a Nation,The Birth of a Nation,1915,195.0,"Drama,War",6.2,25475,Naissance d'une nation,1,nm0001273,actress,"[""Elsie - Stoneman's Daughter""]",Lillian Gish,1893,1993,"actress,writer,soundtrack","tt0048424,tt0094315,tt0019585,tt0038499"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27075,tt9907782,The Cursed,Eight for Silver,2021,111.0,"Fantasy,Horror,Mystery",6.2,13097,Eight for Silver,2,nm0717709,actress,"[""Isabelle Laurent""]",Kelly Reilly,1977,\N,"actress,producer","tt0988045,tt1020530,tt0414387,tt1907668"
27076,tt9907782,The Cursed,Eight for Silver,2021,111.0,"Fantasy,Horror,Mystery",6.2,13097,Eight for Silver,3,nm0677944,actor,"[""Seamus Laurent""]",Alistair Petrie,1970,\N,"actor,soundtrack","tt3748528,tt0200465,tt1979320,tt1976009"
27077,tt9907782,The Cursed,Eight for Silver,2021,111.0,"Fantasy,Horror,Mystery",6.2,13097,Eight for Silver,5,nm1193346,director,\N,Sean Ellis,1970,\N,"director,writer,producer","tt1845838,tt4190530"
27078,tt9907782,The Cursed,Eight for Silver,2021,111.0,"Fantasy,Horror,Mystery",6.2,13097,Eight for Silver,1,nm2933542,actor,"[""John McBride""]",Boyd Holbrook,1981,\N,"actor,writer,producer","tt1517260,tt8110640,tt3829266,tt3315342"


In [24]:
dfgroup = nconstmovs.groupby(['tconst']).agg(
    startYear=('startYear', 'first'),
    nconst=('nconst', lambda x: list(x)),
    runtimeMinutes=('runtimeMinutes', 'first'),
    genres=('genres', 'first'),
    averageRating=('averageRating', 'first'),
    numVotes=('numVotes', 'first'),
    title=('title', 'first')
).reset_index()

In [25]:
dfgroup.sort_values(by="averageRating",ascending=False)

,tconst,startYear,nconst,runtimeMinutes,genres,averageRating,numVotes,title
1270,tt0111161,1994,"[Morgan Freeman, Bob Gunton, Frank Darabont, T...",142.0,Drama,9.3,2738697,Les Évadés
507,tt0068646,1972,"[Marlon Brando, Francis Ford Coppola, James Ca...",175.0,"Crime,Drama",9.2,1904482,Le Parrain
1217,tt0108052,1993,"[Steven Spielberg, Ben Kingsley, Liam Neeson, ...",195.0,"Biography,Drama,History",9.0,1382321,La liste de Schindler
237,tt0050083,1957,"[Henry Fonda, Sidney Lumet, Lee J. Cobb, John ...",96.0,"Crime,Drama",9.0,810614,12 Hommes en colère
2236,tt0468569,2008,"[Michael Caine, Christian Bale, Heath Ledger, ...",152.0,"Action,Crime,Drama",9.0,2711731,The Dark Knight : Le Chevalier noir
...,...,...,...,...,...,...,...,...
4420,tt3678938,2014,"[Ashutosh Rana, Alia Bhatt, Varun Dhawan, Sidh...",133.0,"Comedy,Drama,Romance",6.0,13462,La mariée d'Humpty Sharma
4815,tt5539052,2017,"[Jacki Weaver, Jack Black, Jason Schwartzman, ...",95.0,"Biography,Comedy,Drama",6.0,11310,Le roi de la polka
2675,tt1078912,2009,"[Ben Stiller, Hank Azaria, Owen Wilson, Shawn ...",105.0,"Adventure,Comedy,Family",6.0,203590,La Nuit au musée 2
4441,tt3741700,2019,"[Ken Watanabe, Kyle Chandler, Vera Farmiga, Mi...",132.0,"Action,Adventure,Fantasy",6.0,189927,Godzilla II : Roi des monstres


In [79]:
dfdummies = dfgroup["genres"].str.get_dummies(sep=",")

In [80]:
dfdummies2 = algotest2["genres"].str.get_dummies(sep=",")

In [81]:
dfdummies2

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27056,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27062,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
27066,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27071,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
dfalgotest_act = pd.concat([dfdummies2,
                            algotest2],
                            axis=1)

In [83]:
dfalgotest_act

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,nm9602952,nm9665672,nm9760132,nm9858131,nm9862858,nm9877392,nm9903838,nm9912940,nm9913973,nm9930181
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27056,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27062,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27066,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27071,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
dfalgotest_act2 = pd.concat([dfgroup,
                             dfalgotest_act,
                             ],axis=1)

In [11]:
test = pd.concat([dfgroup,
          dfdummies],
          axis=1)

In [12]:
test

,tconst,startYear,nconst,runtimeMinutes,genres,averageRating,numVotes,title,Action,Adventure,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0004972,1915,"[nm0000428, nm0910400, nm0178270, nm0550615, n...",195.0,"Drama,War",6.2,25475,Naissance d'une nation,0,0,...,0,0,0,0,0,0,0,0,1,0
1,tt0006864,1916,"[nm0366008, nm0000428, nm0550615, nm0001273, n...",163.0,"Drama,History",7.7,16121,Intolérance,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0012349,1921,"[nm0000122, nm0701012, nm0074788, nm0080930, n...",68.0,"Comedy,Drama,Family",8.3,129088,Le Kid,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0013257,1922,"[nm0690702, nm0159725, nm0159770, nm0669922]",91.0,"Documentary,Fantasy,Horror",7.6,15426,La sorcellerie à travers les âges,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0013427,1922,"[nm1354245, nm0638725, nm0192066, nm0020180, n...",78.0,Documentary,7.6,12886,Nanouk l'Esquimau,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,tt9866072,2020,"[nm0004920, nm0925870, nm0155693, nm0731075, n...",104.0,"Comedy,Romance",6.1,71605,Holidate,0,0,...,0,0,0,0,1,0,0,0,0,0
5419,tt9883832,2022,[nm0605137],135.0,"Biography,Documentary,History",7.7,11140,Moonage Daydream,0,0,...,0,0,0,0,0,0,0,0,0,0
5420,tt9893250,2020,"[nm0001848, nm0683253, nm0227759, nm2128335, n...",118.0,"Comedy,Crime,Drama",6.3,137616,I Care a Lot,0,0,...,0,0,0,0,0,0,0,0,0,0
5421,tt9900782,2019,"[nm1230844, nm1912683, nm6998719, nm10183124, ...",145.0,"Action,Adventure,Crime",8.4,35885,Kaithi,1,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dfdummies2 = test['nconst'].str.get_dummies(sep=",").add_prefix('nconst_')

In [21]:
dfdummies2

,nconst_ 'nm0000001',nconst_ 'nm0000002',nconst_ 'nm0000002'],nconst_ 'nm0000003',nconst_ 'nm0000004'],nconst_ 'nm0000005',nconst_ 'nm0000006',nconst_ 'nm0000006'],nconst_ 'nm0000007',nconst_ 'nm0000008',...,nconst_['nm8923002',nconst_['nm8927531',nconst_['nm8958770',nconst_['nm9176319',nconst_['nm9242868',nconst_['nm9393813',nconst_['nm9556770',nconst_['nm9576872',nconst_['nm9654611',nconst_['nm9665672'
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
dfalgo = pd.concat([test,
          dfdummies2],
          axis=1)

In [5]:
dfalgotest = pd.read_csv("dfalgotest.csv")

In [6]:
dfalgotest.index = dfalgotest["title"]

In [113]:
algotest3.index = algotest3["title"]

In [8]:
dfalgotest.head(10)

,title,tconst,startYear,nconst,runtimeMinutes,genres,averageRating,numVotes,title.1,Action,...,nconst_['nm8923002',nconst_['nm8927531',nconst_['nm8958770',nconst_['nm9176319',nconst_['nm9242868',nconst_['nm9393813',nconst_['nm9556770',nconst_['nm9576872',nconst_['nm9654611',nconst_['nm9665672'
title,,,,,,,,,,,,,,,,,,,,,
Naissance d'une nation,Naissance d'une nation,tt0004972,1915,"['nm0000428', 'nm0910400', 'nm0178270', 'nm055...",195.0,"Drama,War",6.2,25475,Naissance d'une nation,0,...,0,0,0,0,0,0,0,0,0,0
Intolérance,Intolérance,tt0006864,1916,"['nm0366008', 'nm0000428', 'nm0550615', 'nm000...",163.0,"Drama,History",7.7,16121,Intolérance,0,...,0,0,0,0,0,0,0,0,0,0
Le Kid,Le Kid,tt0012349,1921,"['nm0000122', 'nm0701012', 'nm0074788', 'nm008...",68.0,"Comedy,Drama,Family",8.3,129088,Le Kid,0,...,0,0,0,0,0,0,0,0,0,0
La sorcellerie à travers les âges,La sorcellerie à travers les âges,tt0013257,1922,"['nm0690702', 'nm0159725', 'nm0159770', 'nm066...",91.0,"Documentary,Fantasy,Horror",7.6,15426,La sorcellerie à travers les âges,0,...,0,0,0,0,0,0,0,0,0,0
Nanouk l'Esquimau,Nanouk l'Esquimau,tt0013427,1922,"['nm1354245', 'nm0638725', 'nm0192066', 'nm002...",78.0,Documentary,7.6,12886,Nanouk l'Esquimau,0,...,0,0,0,0,0,0,0,0,0,0
Nosferatu le vampire,Nosferatu le vampire,tt0013442,1922,"['nm0903194', 'nm0003638', 'nm0775659', 'nm077...",94.0,"Fantasy,Horror",7.9,100603,Nosferatu le vampire,0,...,0,0,0,0,0,0,0,0,0,0
Les lois de l'hospitalité,Les lois de l'hospitalité,tt0014341,1923,"['nm0090007', 'nm0000036', 'nm0444172', 'nm084...",65.0,"Comedy,Romance,Thriller",7.8,11851,Les lois de l'hospitalité,0,...,0,0,0,0,0,0,0,0,0,0
Monte là-dessus!,Monte là-dessus!,tt0014429,1923,"['nm0516001', 'nm0628345', 'nm0205192', 'nm094...",74.0,"Action,Comedy,Thriller",8.1,21620,Monte là-dessus!,1,...,0,0,0,0,0,0,0,0,0,0
Faust,Faust,tt0016847,1926,"['nm0723801', 'nm0417837', 'nm0252472', 'nm000...",107.0,"Drama,Fantasy,Horror",8.1,15873,Faust,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = dfalgotest.select_dtypes("number").drop(["startYear"],axis=1)
y = dfalgotest["title"]

modelNN = KNeighborsClassifier().fit(X,y)

In [324]:
algotest3.to_csv("dfalgotest2.csv")

In [1]:
algotest3 = pd.read_csv(r"C:\Users\Tony\Desktop\PROJET2\dfalgotest2.csv")

NameError: name 'pd' is not defined

In [3]:
actors = pd.read_csv(r"C:\Users\Tony\Desktop\PROJET2\Avec_nconst.csv")

test = actors[(actors['startYear'] >= 1993) & (actors['startYear'] <= 2023) & (actors["category"]=="director")]
test = test.filter(["nconst", "primaryName", "startYear",])
top_director = test['primaryName'].value_counts().head(10)
top_director

Steven Spielberg     18
Steven Soderbergh    17
Richard Linklater    15
Guy Ritchie          13
Quentin Tarantino    12
Ron Howard           12
Woody Allen          12
Joel Coen            11
Luc Besson           11
Lasse Hallström      11
Name: primaryName, dtype: int64

In [4]:
test = actors[(actors['startYear'] >= 1993) & (actors['startYear'] <= 2023) & (actors["category"]=="actor")]
test = test.filter(["nconst", "primaryName", "startYear",])
top_actor = test['primaryName'].value_counts().head(10)
top_actor

Samuel L. Jackson      34
Ryan Reynolds          31
Brad Pitt              29
Matt Damon             28
Matthew McConaughey    27
Liam Neeson            27
Johnny Depp            27
Colin Farrell          26
Mark Wahlberg          26
Jude Law               26
Name: primaryName, dtype: int64

In [12]:
test = actors[(actors['startYear'] >= 1993) & (actors['startYear'] <= 2023) & (actors["category"].str.contains("actor|actress"))]
actoress = test["category"].value_counts()

In [5]:
test = actors[(actors['startYear'] >= 1993) & (actors['startYear'] <= 2023) & (actors["category"]=="actress")]
test = test.filter(["nconst", "primaryName", "startYear",])
top_actress = test['primaryName'].value_counts().head(10)
top_actress

Cate Blanchett        29
Nicole Kidman         28
Julianne Moore        28
Rachel Weisz          23
Naomi Watts           23
Scarlett Johansson    22
Keira Knightley       22
Charlize Theron       21
Amy Adams             21
Kate Winslet          20
Name: primaryName, dtype: int64

In [12]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=False,
                    shared_yaxes=False, horizontal_spacing=0,
                    subplot_titles=("Acteurs", "Actrices"))

top_actor = top_actor.sort_values(ascending=True)
fig.append_trace(go.Bar(x=top_actor.values,
                        y=top_actor.index,
                     textposition='inside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker=dict(
                    color=px.colors.sequential.Viridis)), 
                     1, 1)

top_actress = top_actress.sort_values(ascending=True)
fig.append_trace(go.Bar(x=top_actress.values,
                        y=top_actress.index,
                     textposition='outside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker=dict(
                    color=px.colors.sequential.Viridis),
                     yaxis='y2'), 
                     1, 2)

fig.update_layout(
    yaxis=dict(),
    yaxis2=dict(overlaying='y', side='right'))

fig.update_xaxes(showticklabels=True,title_text="Male", row=1, col=1, autorange='reversed')
fig.update_xaxes(showticklabels=True,title_text="Female", row=1, col=2)

fig.update_layout(title_text="Top 10 Actor/Actress last 30 years", 
                  width=1000, 
                  height=500,
                  title_x=0.5)

fig.show()

In [28]:
import plotly.graph_objects as go

labels = actoress.index
values = actoress.values
colors = ["Gold","Indigo"]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)
                    ])
fig.update_traces(marker=dict(
                    colors=colors, line=dict(color='#000000', width=1)))
fig.show()

In [13]:
fig = go.Figure(go.Bar(x=top_director.index, y=top_director.values, marker=dict(
                    color=px.colors.sequential.Viridis[::-1])))
fig.update_layout(title='Top 10 Directors in the last 30 years', 
                  width=1000, 
                  height=500,
                  title_x=0.5)
fig.show()

In [30]:
testgenres = pd.read_csv(r"C:\Users\Tony\Desktop\PROJET2\Avec_tconst.csv",low_memory=False)

In [37]:
testgenres = testgenres[testgenres['startYear'] != '\\N']
testgenres['startYear'] = testgenres['startYear'].astype(int)

C:\Users\Tony\AppData\Local\Temp\ipykernel_4136\2942505301.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
testgenres = testgenres[(testgenres['startYear'] >= 1993) & (testgenres['startYear'] <= 2023)]

In [40]:
genresiwant = testgenres["genres"].str.get_dummies(sep=",")

In [66]:
bestgenres = pd.concat([testgenres,
                       genresiwant],
                       axis=1)

In [112]:
columns=["averageRating","numVotes"]
bestmov1 = bestgenres.filter(["title","averageRating","numVotes",'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',         
                    'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
                    'History', 'Horror', 'Music', 'Musical', 'Mystery',
                    'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']).sort_values(by="numVotes",ascending=False).head(100)

In [113]:
bestmov2 = bestmov1.drop(["averageRating","numVotes"],axis=1).head(100).sum()

In [122]:
fig5 = go.Figure(data=go.Scatter(
    x=bestmov1['averageRating'],
    y=list(bestmov2.index),
    mode='markers',
    marker=dict(
        color=bestmov1['numVotes'],
        size=bestmov1['numVotes'],
        colorscale='Viridis',
        showscale=True
    ),
    text=bestmov2.index
))

fig5.update_layout(
    title='Top Rated Movies by Genre',
    xaxis=dict(title='Genre'),
    yaxis=dict(title='Average Rating'),
    hovermode='closest'
)

fig5.show()


In [124]:
data = []
for genre in bestmov1.index:
    scatter = go.Scatter(
        x=bestmov1.loc[genre, 'averageRating'],
        y=[genre] * len(bestmov1.loc[genre]),
        mode='markers',
        marker=dict(
            color=bestmov1.loc[genre, 'numVotes'],
            size=bestmov1.loc[genre, 'numVotes'],
            colorscale='Viridis',
            showscale=True
        ),
        name=genre
    )
    data.append(scatter)

fig5 = go.Figure(data=data)

fig5.update_layout(
    title='Top Rated Movies by Genre',
    xaxis=dict(title='Average Rating'),
    yaxis=dict(title='Genre', type='category'),
    hovermode='closest'
)

fig5.show()



ValueError: 
    Invalid value of type 'numpy.float64' received for the 'x' property of scatter
        Received value: 9.3

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [62]:
genresiwantsum = testgenres["genres"].str.get_dummies(sep=",").sum()

In [45]:
totalgenres = genresiwant.sum().sort_values(ascending=False).head(10)

In [61]:
fig4 = go.Figure(go.Treemap(
    values=totalgenres.values,
    labels=totalgenres.index,
    parents=["Genres"] * len(totalgenres.index),
        textinfo = "label+value+percent root",
        textposition='middle center',
        hoverinfo = 'skip',
        texttemplate='<b>%{label}</b><br>%{value} films<br>Ratio : %{percentRoot}',
        insidetextfont=dict(size=16),           
        marker=dict(
            colorscale='Viridis'),
            ))
fig4.update_layout(margin = dict(t=0, l=0, r=0, b=0),                 
    height=630,
    )


In [366]:
X2 = algotest3.select_dtypes("number").drop(["numVotes","startYear","averageRating","runtimeMinutes",'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',         
                    'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
                    'History', 'Horror', 'Music', 'Musical', 'Mystery',
                    'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western'],axis=1)

scaler = StandardScaler()

scaler.fit(X2)

scaled_X = scaler.transform(X2)

modelNN2 = NearestNeighbors(n_neighbors=9).fit(X2)

In [367]:
filmchoisi2 = X2.loc["John Wick: Chapitre 4"]
filmchoisi2_array = np.array(filmchoisi2)
filmchoisi2_reshaped = filmchoisi2_array.reshape(1,-1)

modelNN2.kneighbors(filmchoisi2_reshaped,
                    n_neighbors=11)

c:\Users\Tony\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1874, 4843, 2025, 4300, 1618, 4511, 2580, 1635, 4152,  992, 3057]],
       dtype=int64))

In [368]:
neigh_dist, neigh_index = modelNN2.kneighbors(
    filmchoisi2_reshaped,
    n_neighbors=11
)

c:\Users\Tony\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [369]:
neigh_index2 = neigh_index[0][1:]
voisins = pd.DataFrame(algotest3.iloc[neigh_index2])
voisins


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,title,ordering,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,,
John Wick Parabellum,tt6146586,John Wick: Chapter 3 - Parabellum,John Wick: Chapter 3 - Parabellum,2019,130.0,"Action,Crime,Thriller",7.4,386746,John Wick Parabellum,1,...,0,0,0,0,0,0,0,1,0,0
A Scanner Darkly,tt0405296,A Scanner Darkly,A Scanner Darkly,2006,100.0,"Animation,Comedy,Crime",7.0,114310,A Scanner Darkly,1,...,0,0,0,0,0,0,0,0,0,0
The Whole Truth,tt3503406,The Whole Truth,The Whole Truth,2016,93.0,"Crime,Drama,Mystery",6.2,30807,The Whole Truth,1,...,0,0,0,1,0,0,0,0,0,0
Matrix Reloaded,tt0234215,The Matrix Reloaded,The Matrix Reloaded,2003,138.0,"Action,Sci-Fi",7.2,607653,Matrix Reloaded,1,...,0,0,0,0,0,1,0,0,0,0
John Wick 2,tt4425200,John Wick: Chapter 2,John Wick: Chapter 2,2017,122.0,"Action,Crime,Thriller",7.4,473590,John Wick 2,1,...,0,0,0,0,0,0,0,1,0,0
John Wick: Chapitre 4,tt10366206,John Wick: Chapter 4,John Wick: Chapter 4,2023,169.0,"Action,Crime,Thriller",8.2,146278,John Wick: Chapitre 4,1,...,0,0,0,0,0,0,0,1,0,0
Matrix Revolutions,tt0242653,The Matrix Revolutions,The Matrix Revolutions,2003,129.0,"Action,Sci-Fi",6.7,524748,Matrix Revolutions,1,...,0,0,0,0,0,1,0,0,0,0
John Wick,tt2911666,John Wick,John Wick,2014,101.0,"Action,Crime,Thriller",7.4,689444,John Wick,1,...,0,0,0,0,0,0,0,1,0,0
L'Excellente Aventure de Bill & Ted,tt0096928,Bill & Ted's Excellent Adventure,Bill & Ted's Excellent Adventure,1989,90.0,"Adventure,Comedy,Music",6.9,135600,L'Excellente Aventure de Bill & Ted,1,...,0,1,0,0,0,0,0,0,0,0
